# Automobile Dataset Description

#### This dataset consist of data From 1985 Ward's Automotive Yearbook.

#### This data set consists of three types of entities: 
 - (a) The specification of an auto in terms of various characteristics
 - (b) Its assigned insurance risk rating
 - (c) Its normalized losses in use as compared to other cars. 

#### The second rating corresponds to the degree to which the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price. Then if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale. Actuarians call this process "symboling". A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.

#### The third factor is the relative averages loss payment per insured vehicle year. This value is normalized for all autos within a particular size classification (two-door small, station wagons, sports/speciality, etc...), and represents the averages loss per car per year.

## Importing the Libraries

In [1]:
#!pip install pandas seaborn matplotlib scikit-learn

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
#from pyspark.sql import SparkSession
import uuid
import os
import shutil
import pickle
from datetime import datetime
from subprocess import run, Popen, PIPE
#from pyspark import SparkContext, SparkConf
#from pyspark.sql import DataFrame, SparkSession, Window
#from pyspark.sql.functions import col, expr, monotonically_increasing_id, row_number,current_timestamp
from typing import  List

## Loading the Dataset from Azure

In [3]:
automobile = pd.read_csv("https://raw.githubusercontent.com/katonic-dev/Examples/master/data/automobile_data.csv")
automobile.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [4]:
automobile.dtypes

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
dtype: object

In [5]:
automobile.keys()

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')

##### Shape of the data set

In [6]:
print(automobile.shape)

(205, 26)


##### Describe the stats about the data

In [7]:
automobile.describe()

,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


##### Checking for the null values

In [8]:
# Number of missing in each column
missing = pd.DataFrame(automobile.isnull().sum()).rename(columns = {0: 'total'})

# Create a percentage missing
missing['percent'] = missing['total'] / len(automobile)

missing.sort_values('percent', ascending = False).head(10)

,total,percent
symboling,0,0.0
normalized-losses,0,0.0
highway-mpg,0,0.0
city-mpg,0,0.0
peak-rpm,0,0.0
horsepower,0,0.0
compression-ratio,0,0.0
stroke,0,0.0
bore,0,0.0
fuel-system,0,0.0


In [9]:
automobile.isnull().sum()

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64

##### Find out number of records having '?' value for normalized losses

In [10]:
automobile['normalized-losses'].loc[automobile['normalized-losses'] == '?'].count()

41

##### Setting the missing value to mean of normalized losses and conver the datatype to integer


In [11]:
nl = automobile['normalized-losses'].loc[automobile['normalized-losses'] != '?']
nlmean = nl.astype(str).astype(int).mean()
automobile['normalized-losses'] = automobile['normalized-losses'].replace('?',nlmean).astype(int)
automobile['normalized-losses'].head()

0    122
1    122
2    122
3    164
4    164
Name: normalized-losses, dtype: int64

##### Find out the number of values which are not numeric


In [12]:
automobile['price'].str.isnumeric().value_counts()

True     201
False      4
Name: price, dtype: int64

##### List out the values which are not numeric


In [13]:
automobile['price'].loc[automobile['price'].str.isnumeric() == False]

9      ?
44     ?
45     ?
129    ?
Name: price, dtype: object

##### Setting the missing value to mean of price and convert the datatype to integer


In [14]:
price = automobile['price'].loc[automobile['price'] != '?']
pmean = price.astype(str).astype(int).mean()
automobile['price'] = automobile['price'].replace('?',pmean).astype(int)
automobile['price'].head()

0    13495
1    16500
2    16500
3    13950
4    17450
Name: price, dtype: int64

##### Checking the numberic and replacing with mean value and conver the datatype to integer for horsepower


In [15]:
automobile['horsepower'].str.isnumeric().value_counts()
horsepower = automobile['horsepower'].loc[automobile['horsepower'] != '?']
hpmean = horsepower.astype(str).astype(int).mean()
automobile['horsepower'] = automobile['horsepower'].replace('?',pmean).astype(int)

##### Checking the outlier of horsepower


In [16]:
automobile.loc[automobile['horsepower'] > 10000]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
130,0,122,renault,gas,std,four,wagon,fwd,front,96.1,...,132,mpfi,3.46,3.9,8.7,13207,?,23,31,9295
131,2,122,renault,gas,std,two,hatchback,fwd,front,96.1,...,132,mpfi,3.46,3.9,8.7,13207,?,23,31,9895


##### Excluding the outlier data for horsepower


In [17]:
automobile[np.abs(automobile.horsepower-automobile.horsepower.mean())<=(3*automobile.horsepower.std())]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,122,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.4,23.0,106,4800,26,27,22470


##### Find out the number of invalid value


In [18]:
automobile['bore'].loc[automobile['bore'] == '?']

55    ?
56    ?
57    ?
58    ?
Name: bore, dtype: object

##### Replace the non-numeric value to null and conver the datatype


In [19]:
automobile['bore'] = pd.to_numeric(automobile['bore'],errors='coerce')
automobile.dtypes

symboling              int64
normalized-losses      int64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke                object
compression-ratio    float64
horsepower             int64
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                  int64
dtype: object

##### Replace the non-number value to null and convert the datatype


In [20]:
automobile['stroke'] = pd.to_numeric(automobile['stroke'],errors='coerce')
automobile.dtypes

symboling              int64
normalized-losses      int64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower             int64
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                  int64
dtype: object

##### Convert the non-numeric data to null and convert the datatype


In [21]:
automobile['peak-rpm'] = pd.to_numeric(automobile['peak-rpm'],errors='coerce')
automobile.dtypes

symboling              int64
normalized-losses      int64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower             int64
peak-rpm             float64
city-mpg               int64
highway-mpg            int64
price                  int64
dtype: object

### Label Encoder
##### In machine learning, we usually deal with datasets which contains multiple labels in one or more than one columns. These labels can be in the form of words or numbers. Label Encoding refers to converting the labels info the labels into numeric form so as to convert it into the machine-readable form.

In [22]:
le = LabelEncoder()
automobile['make'] = le.fit_transform(automobile['make'])
automobile['fuel-type'] = le.fit_transform(automobile['fuel-type'])
automobile['aspiration'] = le.fit_transform(automobile['aspiration'])
automobile['num-of-doors'] = le.fit_transform(automobile['num-of-doors'])
automobile['body-style'] = le.fit_transform(automobile['body-style'])
automobile['drive-wheels'] = le.fit_transform(automobile['drive-wheels'])
automobile['engine-location'] = le.fit_transform(automobile['engine-location'])
automobile['engine-type'] = le.fit_transform(automobile['engine-type'])
automobile['num-of-cylinders'] = le.fit_transform(automobile['num-of-cylinders'])
automobile['fuel-system'] = le.fit_transform(automobile['fuel-system'])
automobile.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,122,0,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000.0,21,27,13495
1,3,122,0,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000.0,21,27,16500
2,1,122,0,1,0,2,2,2,0,94.5,...,152,5,2.68,3.47,9.0,154,5000.0,19,26,16500
3,2,164,1,1,0,1,3,1,0,99.8,...,109,5,3.19,3.40,10.0,102,5500.0,24,30,13950
4,2,164,1,1,0,1,3,0,0,99.4,...,136,5,3.19,3.40,8.0,115,5500.0,18,22,17450


##### Converting the datatype

In [23]:
automobile['wheel-base'] = automobile['wheel-base'].astype(int)
automobile['height'] = automobile['height'].astype(int)
automobile['length'] = automobile['length'].astype(int)
automobile['width'] = automobile['width'].astype(int)
automobile['compression-ratio'] = automobile['compression-ratio'].astype(int)

##### Filling the null value with mean and then change the datatype for stroke

In [24]:
automobile['stroke'] = automobile['stroke'].fillna(automobile['stroke'].mean())
automobile['stroke'] = automobile['stroke'].astype(int)

##### Filling the null value with mean and then change the datatype for peak-rpm

In [25]:
automobile['peak-rpm'] = automobile['peak-rpm'].fillna(automobile['peak-rpm'].mean())
automobile['peak-rpm'] = automobile['peak-rpm'].astype(int)

##### Filling the null value with mean and then change the datatype for bore

In [26]:
automobile['bore'] = automobile['bore'].fillna(automobile['bore'].mean())
automobile['bore'] = automobile['bore'].astype(int)

In [27]:
automobile.dtypes

symboling            int64
normalized-losses    int64
make                 int64
fuel-type            int64
aspiration           int64
num-of-doors         int64
body-style           int64
drive-wheels         int64
engine-location      int64
wheel-base           int64
length               int64
width                int64
height               int64
curb-weight          int64
engine-type          int64
num-of-cylinders     int64
engine-size          int64
fuel-system          int64
bore                 int64
stroke               int64
compression-ratio    int64
horsepower           int64
peak-rpm             int64
city-mpg             int64
highway-mpg          int64
price                int64
dtype: object

##### Remove the records which are having the value '?'


In [28]:
automobile['num-of-doors'].loc[automobile['num-of-doors'] == '?']
automobile = automobile[automobile['num-of-doors'] != '?']
automobile['num-of-doors'].loc[automobile['num-of-doors'] == '?']

Series([], Name: num-of-doors, dtype: int64)

### Correlation Matrix
##### A correlation matrix is a table showing correlation coefficients between variables. Each cell in the table showns the correlation between two variables. The values in the range of -1 to 1. If two variables have high correlations, we can neglect one variable from those two.

In [29]:
automobile.corr()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
symboling,1.000000,4.651899e-01,-0.118794,0.194311,-0.059866,0.645492,-0.596135,-0.041671,2.124709e-01,-0.533322,...,-0.105790,0.091163,-0.122112,-0.076884,-0.171731,0.015430,0.273678,-0.035823,0.034606,-0.082201
normalized-losses,0.465190,1.000000e+00,-0.250975,0.101437,-0.006823,0.336885,-0.250893,0.311032,-2.918301e-17,-0.059610,...,0.110997,0.228035,-0.109550,0.090908,-0.115825,0.006224,0.237748,-0.218749,-0.178221,0.133999
make,-0.118794,-2.509755e-01,1.000000,-0.113191,0.054265,-0.127416,0.089494,-0.004317,5.460831e-02,0.084751,...,-0.070918,0.146624,0.258004,-0.123254,0.136030,0.058665,-0.218347,0.053642,0.050022,-0.161471
fuel-type,0.194311,1.014370e-01,-0.113191,1.000000,-0.401397,0.207665,-0.147853,-0.132257,4.006951e-02,-0.304407,...,-0.069594,0.041529,-0.064795,-0.103911,-0.982732,0.037675,0.477058,-0.255963,-0.191392,-0.110207
aspiration,-0.059866,-6.822700e-03,0.054265,-0.401397,1.000000,-0.066737,0.063028,0.066465,-5.719146e-02,0.255692,...,0.108217,0.288086,0.166830,0.073610,0.294949,-0.039211,-0.183626,-0.202362,-0.254416,0.177285
num-of-doors,0.645492,3.368845e-01,-0.127416,0.207665,-0.066737,1.000000,-0.670198,0.112398,1.365776e-01,-0.415741,...,-0.005701,-0.000323,-0.157658,-0.046524,-0.188059,0.018703,0.233903,-0.000142,0.021628,-0.031934
body-style,-0.596135,-2.508926e-01,0.089494,-0.147853,0.063028,-0.670198,1.000000,-0.155745,-2.770093e-01,0.404010,...,-0.073352,-0.065079,0.110676,0.059036,0.137310,0.039949,-0.109414,0.031697,-0.007170,-0.072677
drive-wheels,-0.041671,3.110317e-01,-0.004317,-0.132257,0.066465,0.112398,-0.155745,1.000000,1.478645e-01,0.462127,...,0.524307,0.424686,0.181554,0.082601,0.128359,-0.042627,-0.039719,-0.449581,-0.452220,0.576867
engine-location,0.212471,-2.918301e-17,0.054608,0.040070,-0.057191,0.136578,-0.277009,0.147865,1.000000e+00,-0.187241,...,0.196826,0.105971,0.043322,-0.320364,-0.025977,-0.002373,0.198401,-0.153487,-0.102026,0.331013
wheel-base,-0.533322,-5.960954e-02,0.084751,-0.304407,0.255692,-0.415741,0.404010,0.462127,-1.872415e-01,1.000000,...,0.570816,0.389963,0.323980,0.148477,0.249054,-0.026748,-0.356787,-0.474734,-0.547507,0.585736


## Model Training

In [30]:
X = automobile.drop(columns = ['price'])
Y = automobile['price']

### Logistic Regression

In [31]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, Y, test_size=0.30)

In [32]:
logreg = LogisticRegression(solver='lbfgs', max_iter=110)
lr = logreg.fit(X_train_lr, y_train_lr)
acc_log = round(logreg.score(X_train_lr, y_train_lr) * 100, 2)
acc_log

60.14

### Random Forest

In [33]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, Y, test_size=0.30)

In [34]:
random_forest = RandomForestClassifier(n_estimators=100)
rf_auc_result = random_forest.fit(X_train_rf, y_train_rf)
acc_random_forest = round(random_forest.score(X_train_rf, y_train_rf) * 100, 2)
acc_random_forest

97.9

### Gaussian Naive Bayes

In [35]:
X_train_gnb, X_test_gnb, y_train_gnb, y_test_gnb = train_test_split(X, Y, test_size=0.30)

In [36]:
gaussian = GaussianNB()
gnb = gaussian.fit(X_train_gnb, y_train_gnb)
acc_gaussian = round(gaussian.score(X_train_gnb, y_train_gnb) * 100, 2)
acc_gaussian

97.2

### Support Vector Machine

In [37]:
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, Y, test_size=0.30)

In [38]:
linear_svc = SVC(gamma='auto')
svm = linear_svc.fit(X_train_svm, y_train_svm)
acc_linear_svc = round(linear_svc.score(X_train_svm, y_train_svm) * 100, 2)
acc_linear_svc

97.9

### Decision Tree

In [39]:
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.30)

In [40]:
decision_tree = DecisionTreeClassifier()
dt = decision_tree.fit(X_train_dt, y_train_dt)
acc_decision_tree = round(decision_tree.score(X_train_dt, y_train_dt) * 100, 2)
acc_decision_tree

97.2

### Preceptron

In [41]:
X_train_per, X_test_per, y_train_per, y_test_per = train_test_split(X, Y, test_size=0.30)

In [42]:
perceptron = Perceptron()
per = perceptron.fit(X_train_per, y_train_per)
acc_perceptron = round(perceptron.score(X_train_per, y_train_per) * 100, 2)
acc_perceptron

1.4

### K Nearest Neighbour

In [43]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, Y, test_size=0.30)

In [44]:
knearestneighbour = KNeighborsClassifier()
knn = knearestneighbour.fit(X_train_knn, y_train_knn)
acc_knearest = round(knearestneighbour.score(X_train_knn, y_train_knn) * 100, 2)
acc_knearest

19.58

### Stochastic Gradient Descent

In [45]:
X_train_sgd, X_test_sgd, y_train_sgd, y_test_sgd = train_test_split(X, Y, test_size=0.30)

In [46]:
stochasticgrad = SGDClassifier()
sgd = stochasticgrad.fit(X_train_sgd, y_train_sgd)
acc_sgd = round(stochasticgrad.score(X_train_sgd, y_train_sgd) * 100, 2)
acc_sgd

2.8

### Gradient Boosting Classifier

In [47]:
X_train_gbc, X_test_gbc, y_train_gbc, y_test_gbc = train_test_split(X, Y, test_size=0.30)

In [48]:
gradientboostingclassifier = GradientBoostingClassifier()
gbc = gradientboostingclassifier.fit(X_train_gbc, y_train_gbc)
acc_gbc = round(gradientboostingclassifier.score(X_train_gbc, y_train_gbc) * 100, 2)
acc_gbc

96.5

## Evaluating the Models

In [49]:
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Gaussian Naive Bayes', 'Support Vector Machine', 'Decision Tree', 'Preceptron', 'KNearest Neighbour', 'Stochastic Gradient Descent', 'Gradient Boosting Classifier'],
    'Score': [acc_log, acc_random_forest, acc_gaussian, acc_linear_svc, acc_decision_tree, acc_perceptron, acc_knearest, acc_sgd, acc_gbc],
    'Model_abb': [lr, rf_auc_result , gnb, svm, dt, per, knn, sgd, gbc]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df

,Model,Model_abb
Score,,
97.90,Random Forest,"(DecisionTreeClassifier(max_features='auto', r..."
97.90,Support Vector Machine,SVC(gamma='auto')
97.20,Gaussian Naive Bayes,GaussianNB()
97.20,Decision Tree,DecisionTreeClassifier()
96.50,Gradient Boosting Classifier,([DecisionTreeRegressor(criterion='friedman_ms...
60.14,Logistic Regression,LogisticRegression(max_iter=110)
19.58,KNearest Neighbour,KNeighborsClassifier()
2.80,Stochastic Gradient Descent,SGDClassifier()
1.40,Preceptron,Perceptron()


In [50]:
best_model = result_df['Model_abb'].iloc[0]
best_model

RandomForestClassifier()

In [51]:
joblib.dump(best_model, 'model.joblib')

['model.joblib']

In [52]:
Model_job = joblib.load("model.joblib")
Model_job

RandomForestClassifier()

In [53]:
type(Model_job)

sklearn.ensemble._forest.RandomForestClassifier

In [54]:
type(X_test_rf)

pandas.core.frame.DataFrame

In [55]:
X_test_rf

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
57,3,150,8,1,0,2,2,2,0,95,...,6,70,2,3,3,9,101,6000,17,23
127,3,122,15,1,0,2,1,2,1,89,...,3,194,5,3,2,9,207,5900,17,25
96,1,122,12,1,0,1,3,1,0,94,...,2,97,1,3,3,9,69,5200,31,37
87,1,125,11,1,1,1,3,1,0,96,...,2,110,6,3,3,7,116,5500,23,30
86,1,125,11,1,0,1,3,1,0,96,...,2,122,1,3,3,8,88,5000,25,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,1,107,5,1,0,2,3,1,0,96,...,2,110,1,3,3,9,100,5500,25,31
66,0,122,8,0,0,1,3,2,0,104,...,2,134,3,3,3,22,72,4200,31,39
185,2,94,20,1,0,1,3,1,0,97,...,2,109,5,3,3,9,85,5250,27,34
119,1,119,14,1,1,2,2,1,0,93,...,2,98,6,3,3,7,102,5500,24,30


In [59]:
test_row = np.random.randint(0, X_test_rf.shape[0])
test_row

1

In [62]:
pred = X_test_rf.values[test_row].reshape(1, -1)
pred

array([[   3,  122,   15,    1,    0,    2,    1,    2,    1,   89,  168,
          65,   51, 2756,    4,    3,  194,    5,    3,    2,    9,  207,
        5900,   17,   25]])

In [ ]:
Model_job.predict(pred)